# Projected density of states (PDOS)

## Visualizing the WorkGraph Builder



In [1]:
from workgraph_collections.ase.espresso.pdos import pdos_workgraph
from aiida import load_profile
load_profile()

node = pdos_workgraph.node()
node.to_html()

## Visualizing the WorkGraph

In [2]:
from workgraph_collections.ase.espresso.atomization_energy import atomization_energy

wg = atomization_energy()
wg.to_html()

## Example: Silicon

### Prepare the inputs and submit the workflow


In [3]:
from ase.build import bulk
from aiida import load_profile
from copy import deepcopy

from workgraph_collections.ase.espresso.pdos import pdos_workgraph

load_profile()

atoms = bulk('Si')


metadata = {
    "options": {
        'prepend_text' : """eval "$(conda shell.posix hook)"
        conda activate aiida
        export OMP_NUM_THREADS=1
        """,
    }
}
pseudopotentials = {"Si": "Si.pbe-nl-rrkjus_psl.1.0.0.UPF"}
pseudo_dir = "/home/xing/data/ase/espresso_pseudo"
# pseudo_dir = "/home/wang_x3/datas/pseudos/psl"
scf_input_data = {
    "control": {"calculation": "scf",
                },
    "system": {"ecutwfc": 30, "ecutrho": 240,
               "occupations": "smearing",
                "degauss": 0.01,
                "smearing": "cold"
                },
}
nscf_input_data = deepcopy(scf_input_data)
nscf_input_data["control"].update({"calculation": "nscf",
                                   })
inputs = {
    "scf": {
        "input_data": scf_input_data,
        "kpts": (4, 4, 4),
        "computer": "localhost",
        "metadata": metadata
    },
    "nscf": {
        "input_data": nscf_input_data,
        "kpts": (6, 6, 6),
        "computer": "localhost",
        "metadata": metadata
    },
    "dos": {
        "input_data": {},
        "computer": "localhost",
        "metadata": metadata
    }
}
#------------------------- Set the inputs -------------------------
wg = pdos_workgraph(atoms=atoms,
                    pw_command="mpirun -np 1 pw.x",
                    dos_command="mpirun -np 1 dos.x",
                    projwfc_command="mpirun -np 1 projwfc.x",
                    pseudopotentials=pseudopotentials,
                    pseudo_dir=pseudo_dir,
                    inputs=inputs)
#------------------------- Submit the calculation -------------------
# wg.run()
wg.submit(wait=True, timeout=200)
#------------------------- Print the output -------------------------
print('dos: \n {}'.format(wg.nodes['dos'].outputs["result"].value.value))
print('pdos: \n {}'.format(wg.nodes['projwfc'].outputs["result"].value.value))



WorkGraph node created, PK: 51472
dos: 
 {'header': array('#  E (eV)   dos(E)     Int dos(E) EFermi =    6.674 eV\n',
      dtype='<U55'), 'dos_energy': array([-6.133, -6.123, -6.113, ..., 16.737, 16.747, 16.757]), 'dos_energy_units': array('eV', dtype='<U2'), 'dos': array([ 2.578e-07,  4.348e-07,  7.253e-07, ..., -5.186e-03, -3.975e-03,
       -3.007e-03]), 'dos_units': array('states/eV', dtype='<U9'), 'integrated_dos': array([2.578e-09, 6.926e-09, 1.418e-08, ..., 1.600e+01, 1.600e+01,
       1.600e+01]), 'integrated_dos_units': array('states', dtype='<U6')}
pdos: 
 None


In [4]:
print('pdos: \n {}'.format(wg.nodes['projwfc'].outputs["result"].value.value))


pdos: 
 None
